In [3]:
import os,shutil,json,wandb,ast
import io as input_output
import datasets,evaluate
import numpy as np
import pandas as pd
from PIL import Image
from pathlib import Path
from tqdm.auto import tqdm
import multiprocessing as mp
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,f1_score
import torch
import torch.nn as nn
import torch.nn.functional as F
from datasets import Dataset, DatasetDict
from torchvision import io, transforms
from torch.utils.data import Dataset, DataLoader, random_split
from datasets import load_dataset
from transformers import Trainer,TrainingArguments, AutoModel, BlipForConditionalGeneration
from transformers import VisionEncoderDecoderModel, ViTFeatureExtractor
from transformers import AutoTokenizer, GPT2Config, default_data_collator
from transformers import VisionEncoderDecoderModel, AutoTokenizer, AutoFeatureExtractor, Seq2SeqTrainer, Seq2SeqTrainingArguments
from transformers import AutoProcessor, AutoModelForCausalLM
from scipy.stats import pearsonr
from textwrap import wrap
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
if torch.cuda.is_available():
    print(f"There are {torch.cuda.device_count()} GPU(s) available.")
    print(f"We will use the GPU: {torch.cuda.get_device_name(0)}")
else:
    print("No GPU available, using the CPU instead.")
torch.cuda.empty_cache()
wandb.login(key="e6e4c8d10c1c73d5cbae270330b5ecb2e2b3944a")


No GPU available, using the CPU instead.


Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.
wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: C:\Users\thomb\_netrc


True

In [4]:
EPOCHS = 2
BATCH_SIZE = 2 #blip > 2, git = 2
# checkpoint = "Salesforce/blip-image-captioning-base"
checkpoint = "microsoft/git-base-textcaps"
OUTPUT_DIR = '/kaggle/tmp/'
OUTPUT_MODEL_NAME = checkpoint
IMG_ENHANCEMENT = ''

In [5]:
processor = AutoProcessor.from_pretrained(checkpoint)
model = AutoModelForCausalLM.from_pretrained(checkpoint)

preprocessor_config.json:   0%|          | 0.00/503 [00:00<?, ?B/s]

C:\Users\thomb\anaconda3\Lib\site-packages\huggingface_hub\file_download.py:157: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\thomb\.cache\huggingface\hub\models--microsoft--git-base-textcaps. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


tokenizer_config.json:   0%|          | 0.00/453 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/2.82k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/707M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/141 [00:00<?, ?B/s]

In [6]:
enhancement_type = f'_{IMG_ENHANCEMENT}' if IMG_ENHANCEMENT != '' else ''
df_train = pd.read_csv(f'/kaggle/input/medic-caption-dataset-exists/radiologytraindata{enhancement_type}.csv', delimiter=',')
df_val = pd.read_csv(f'/kaggle/input/medic-caption-dataset-exists/radiologyvaldata{enhancement_type}.csv', delimiter=',')
df_test = pd.read_csv(f'/kaggle/input/medic-caption-dataset-exists/radiologytestdata.csv', delimiter=',')#, nrows = nRowsRead
print(len(df_train.columns))
print(len(df_train),len(df_val),len(df_test))
df_train = df_train.sample(frac=0.05, random_state=42).reset_index(drop=True)
df_val = df_val.sample(frac=0.05, random_state=42).reset_index(drop=True)
df_test = df_test.sample(frac=0.05, random_state=42).reset_index(drop=True)
print(len(df_train),len(df_val),len(df_test))
df_train.head()

FileNotFoundError: [Errno 2] No such file or directory: '/kaggle/input/medic-caption-dataset-exists/radiologytraindata.csv'